# Running Image Classification on a webcam or video feed

This sample shows you how to run OpenVINO inference engine along with a webcam or video stream inside a notebook. You can see the real-time result at the bottom of this notebook. 

To find out more about the model and the number of classes it supported, please refer to the details in the 001-hello-world notebook.

In [13]:
import numpy as np
import cv2

from IPython import display
from PIL import Image

import json
from openvino.inference_engine import IECore

import matplotlib.pyplot as py
%matplotlib inline

In [14]:
def drawResults(image, results, imagenet_classes, topK=5):
    #draw results
    result_index = np.argmax(results)
    indices = np.argsort(-results[0])
    for i in range(topK):
        result_text = (imagenet_classes[indices[i]][1])
        p_length = float(results[0][indices[i]])
        y = 40*(i+1)
        x = 20
        width = image.shape[1]
        if(p_length>0.15):
            cv2.putText(image, text=result_text, org=(x,y), fontFace=1, 
                fontScale=1.5, color=(0,255,0), thickness=2)
            cv2.line(image, (x, y+5), (int(p_length*width), y+5),(200,200,200),5)
    return image

#This function handles the video stream.
def classifyVideo(VideoIndex=0, scale=0.5):
    ie = IECore()
    net = ie.read_network(
        #model="model/public/mobilenet-v3-large-1.0-224-tf/FP16/mobilenet-v3-large-1.0-224-tf.xml"
        model="../001-hello-world/model/v3-small_224_1.0_float.xml"
    )
    exec_net = ie.load_network(net, "CPU")

    input_key = list(exec_net.input_info)[0]
    output_key = list(exec_net.outputs.keys())[0]

    imagenet_classes = json.loads(open("../001-hello-world/utils/imagenet_class_index.json").read())
    imagenet_classes = {
        int(key) + 1: value for key, value in imagenet_classes.items()
    }
    try:
        cap = cv2.VideoCapture(VideoIndex)
    except:
        print("Cannot Open Device")
        del exec_net
    try:
        ret, frame = cap.read()
        
        while(ret==True):
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if not ret:
                # Release the Video Device if ret is false
                cap.release()
                # Message to be displayed after releasing the device
                print ("Released Video Resource")
                break
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            half_frame = cv2.resize(frame, (0, 0), fx = scale, fy = scale)
            
            # resize to MobileNet image shape
            p_frame = cv2.resize(frame, (224, 224))  
            #convert to RGB color space
            p_frame = cv2.cvtColor(p_frame, cv2.COLOR_BGR2RGB)
            # reshape to network input shape
            p_frame = np.expand_dims(p_frame.transpose(2, 0, 1), 0)  
            
            #run the inference
            results = exec_net.infer(inputs={input_key: p_frame})[output_key]
            
            #draw the results
            half_frame = drawResults(half_frame, results, imagenet_classes)
            
            #convert to jpg for performance results
            _,ret_array = cv2.imencode('.jpg', half_frame) 
            i = display.Image(data=ret_array)            
            
            display.display(i)
            display.clear_output(wait=True)
            
    except KeyboardInterrupt:
        # Release the Video Device
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource from KeyboardInterrupt")
        del exec_net
    pass

### Run the Webcam Feed

Please use the stop button to close off the thread nicely. Otherwise you may need to restart the kernel to unlock the resource.

In [17]:
#try the webcam
classifyVideo(0, 0.5)

Released Video Resource from KeyboardInterrupt


In [18]:
#You can also run this on a pre-recorded video with the following command
#classifyVideo("sample.avi")